In [2]:
import torch
print(f"Is GPU available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")

Is GPU available: True
Number of GPUs: 1
GPU name: NVIDIA GeForce GTX 1060 3GB


/home/melan/miniconda3/envs/byola/lib/python3.10/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 1060 3GB which is of cuda capability 6.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/home/melan/miniconda3/envs/byola/lib/python3.10/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/home/melan/miniconda3/envs/byola/lib/python3.10/site-packages/torch/cuda/__init__.py:326: UserWarning: 
NVIDIA GeForce GTX 1060 3GB with CUDA capability sm_61 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the NVIDIA GeForce GTX 1060 3GB GPU with PyTorch

In [ ]:
# testing librosa if it makes a cochleagram

import librosa
import numpy as np
import matplotlib.pyplot as plt

y, sr = librosa.load("test.wav", sr=None)
# simple cochleagram approximation using STFT magnitude
S = np.abs(librosa.stft(y, n_fft=512, hop_length=256))
plt.imshow(20*np.log10(S+1e-6), origin='lower', aspect='auto')
plt.title("Cochleagram-like representation")
plt.show()

In [5]:
import sys
sys.path.append('../models/byol-a/v2')

from byol_a2.common import load_yaml_config
from byol_a2.augmentations import PrecomputedNorm
from byol_a2.models import AudioNTT2022, load_pretrained_weights
import nnAudio.features
import torchaudio
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg = load_yaml_config('config_v2.yaml')  # Use your config file path

stats = [-9.660292, 4.7219563]
to_melspec = nnAudio.features.MelSpectrogram(
    sr=cfg.sample_rate,
    n_fft=cfg.n_fft,
    win_length=cfg.win_length,
    hop_length=cfg.hop_length,
    n_mels=cfg.n_mels,
    fmin=cfg.f_min,
    fmax=cfg.f_max,
    center=True,
    power=2,
    verbose=False,
)
normalizer = PrecomputedNorm(stats)

model = AudioNTT2022(n_mels=cfg.n_mels, d=cfg.feature_d)
load_pretrained_weights(model, 'AudioNTT2022-BYOLA-64x96d3072.pth')
model.eval()
model.to(device)

/home/melan/supervised-vs-SSL/supervised_vs_ssl/tests/../models/byol-a/v2/byol_a2/common.py:31: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.set_audio_backend("sox_io")
/home/melan/miniconda3/envs/byola/lib/python3.10/site-packages/torchaudio/_internal/module_utils.py:71: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  return func(*args, **kwargs)


AssertionError: 

In [ ]:
import os

wav_dir = '/path/to/your/wav/files'  # Change to your .wav directory
wav_files = [f for f in os.listdir(wav_dir) if f.endswith('.wav')]

for f in wav_files[:5]:  # Run on first 5 files
    wav, sr = torchaudio.load(os.path.join(wav_dir, f))
    assert sr == cfg.sample_rate, "Sample rate mismatch!"
    wav = wav[:, :int(cfg.unit_sec * cfg.sample_rate)]  # Crop to 0.95s if needed
    lms = normalizer((to_melspec(wav) + torch.finfo(torch.float).eps).log())
    lms = lms[:, :96]  # Crop/pad to 96 frames
    if lms.shape[1] < 96:
        pad_width = 96 - lms.shape[1]
        lms = torch.nn.functional.pad(lms, (0, pad_width))
    with torch.no_grad():
        features = model(lms.unsqueeze(0).to(device))
    print(f"File: {f}, Feature shape: {features.shape}")